# Iterators, generator expressions, and generators

## Iterators

In [66]:
nums = [1,2,3]
iter(nums)

In [67]:
nums.__iter__()

In [68]:
nums.__reversed__()

In [69]:
it = iter(nums)
next(it)

1

In [70]:
next(it)

2

In [71]:
next(it)

3

In [72]:
# When used in a loop, StopIteration is swallowed and 
# causes the loop to finish. But with explicit invocation, 
# we can see that once the iterator is exhausted, 
# accessing it raises an exception
try:
    next(it)
except Exception:
    pass
finally:
    pass


## Generator expressions

Generator expression is the basis of __list comprehension__. Generator expression must always be enclosed in parentheses or an expression. If round parentheses are used, then a generator iterator is created. If rectangular parentheses are used, the process is short-circuited and we get a list.

In [73]:
(i for i in nums)

<generator object <genexpr> at 0x10c317308>

In [74]:
[i for i in nums]

[1, 2, 3]

In [75]:
list(i for i in nums)

[1, 2, 3]

In [76]:
# similarly applied to set and dictionary comprehensions

In [77]:
myset = {i for i in nums}
myset

{1, 2, 3}

In [78]:
type(myset)

set

In [79]:
mydict = {i:i**2 for i in nums}
mydict

{1: 1, 2: 4, 3: 9}

In [80]:
type(mydict)

dict

## Generator function

Generator functions are marked with the keyword __yield__.

In [81]:
def f():
    yield 1
    yield 2
    
f()

<generator object f at 0x10c317af0>

In [82]:
gen = f()

In [83]:
next(gen)

1

In [84]:
next(gen)

2

In [85]:
try:
    next(gen)
except Exception:
    pass

In [86]:
def f():
    print('-- start --')
    yield 3
    print('-- middle --')
    yield 4
    print('-- finished --')
    
gen = f() # nothing is printed! the first action will take place after 'next'

In [87]:
next(gen)

-- start --


3

In [88]:
next(gen)

-- middle --


4

In [89]:
try:
    next(gen)
except Exception:
    pass

-- finished --


## Bidirectional communication

In [94]:
import itertools
def g():
    print('-- start --')
    for i in itertools.count():
        print('-- yielding %i --' % i)
        try:
            ans = yield i
        except GeneratorExit:
            print('-- closing --')
            raise
        except Exception as e:
            print('--yield raised %r--' % e)
        else:
            print('--yield returned %s--' % ans)
            
it = g()

-- closing --


In [95]:
next(it)

-- start --
-- yielding 0 --


0

In [96]:
it.send(11)

--yield returned 11--
-- yielding 1 --


1

In [97]:
it.throw(IndexError)

--yield raised IndexError()--
-- yielding 2 --


2

In [98]:
it.close()

-- closing --
